#  import the necessary libraries

In [ ]:
import pandas as pd
import numpy as np
import ast
from sklearn.metrics.pairwise import cosine_similarity


# import the embeddins

In [ ]:
path = 'text_chunks_embeddings_2.csv' #path to the chunks and embedding
df = pd.read_csv(path )

# Convert embedding string → numpy vector
df["vec"] = df["embedding"].apply(
    lambda x: np.array(ast.literal_eval(x), dtype=np.float32)
)


In [ ]:
# calculate the centroids
centroids = (
    df.groupby("category")["vec"]
      .apply(lambda s: np.mean(np.stack(s.values), axis=0))
      .to_dict()
)


# importing the embdding model

In [ ]:
from sentence_transformers import SentenceTransformer
import torch
device = 'cuda' if torch.cuda.is_available() else 'cpu'
embedding_model  = SentenceTransformer( model_name_or_path= 'all-mpnet-base-v2' ,
                                       device = device )

#The retrival funtions

In [ ]:
def retrieve(query_text, model = embedding_model , top_categories=2, top_k=5):

    # 1️⃣ Embed query
    query_vec = model.encode(query_text)
    query_vec = np.array(query_vec, dtype=np.float32)

    # 2️⃣ Compute category similarities
    category_scores = {
        cat: float(
            cosine_similarity(
                query_vec.reshape(1, -1),
                centroid.reshape(1, -1)
            )[0][0]
        )
        for cat, centroid in centroids.items()
    }

    # 3️⃣ Select top N categories
    sorted_categories = sorted(
        category_scores.items(),
        key=lambda x: x[1],
        reverse=True
    )

    selected_categories = [cat for cat, _ in sorted_categories[:top_categories]]

    # 4️⃣ Filter dataset by selected categories
    filtered_df = df[df["category"].isin(selected_categories)].copy()

    # 5️⃣ Compute chunk similarity
    matrix = np.stack(filtered_df["vec"].values)

    scores = cosine_similarity(
        query_vec.reshape(1, -1),
        matrix
    )[0]

    filtered_df["score"] = scores

    # 6️⃣ Select top-k per category
    results = (
        filtered_df
        .sort_values(["category", "score"], ascending=[True, False])
        .groupby("category")
        .head(top_k)
    )

    return results[["text", "department", "category", "score", "source"]], category_scores

# login to hugging face

In [ ]:
import huggingface_hub
from huggingface_hub import login
login() # you needed to pass your hugging face token

#  importing the model

In [ ]:


import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

model_name = "meta-llama/Meta-Llama-3-8B-Instruct"


# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name ,
                                          use_auth_token = True )

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype = torch.float16 ,
    device_map="auto"
)

print("Model Loaded Successfully!")

#  generate the output for the given query

In [ ]:
def generate_response(prompt, max_tokens=512):
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_tokens,
            temperature=0.7,
            top_p=0.9,
            do_sample=True,
            # repetition_penalty=1.12
        )

    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response

# Add the prompt to input text

In [ ]:
def get_prompt( retrived , message , history ) :
  conversation = ""

  for user_msg, bot_msg in history:
      conversation += f"User: {user_msg}\nAssistant: {bot_msg}\n"
  prompt = f"""
  You are an assistant answering questions about Panimalar Engineering Institution.

  Use only the context below.
  Conversation so far:
  { conversation }

  Context:
  {retrived}

  Question:
  {message}

  Answer:
  """
  return prompt


#  chat interface preprocessing funtion

In [ ]:
def chat_interface(message, history):
  results, category_scores = retrieve(message, embedding_model )
  content = "\n\n".join(results["text"].tolist() )
  prompt = get_prompt( content , message  , history )
  result = generate_response( prompt )
  return result[len( prompt)-1 : ]

#  the gradio the user interface

In [ ]:
import gradio as gr



demo = gr.ChatInterface(
    fn=chat_interface,
    title="🎓 College RAG Chatbot (LLaMA 3 7B)",
    description="Ask questions about the college.",
)

demo.launch(share=True , debug=True )

In [ ]:
'hi'